In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import seaborn as sns


## Load and preprocess dataset

In [11]:
# Fetch dataset
phishing_websites = fetch_ucirepo(id=327)
X = phishing_websites.data.features
y = phishing_websites.data.targets

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train_tensor = torch.tensor(X_train.values.astype(float), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.astype(float), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values.astype(float), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.astype(float), dtype=torch.float32)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Define the neural network model

In [12]:
class MalwareDetector(nn.Module):
    def __init__(self):
        super(MalwareDetector, self).__init__()
        self.fc1 = nn.Linear(30, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 16)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

model = MalwareDetector()

## Define training and evaluation functions

In [13]:
def train_model(model, criterion, optimizer, train_loader, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            labels = (labels == 1).float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

## Add noise to model weights

In [14]:
def add_noise_to_weights(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            noise = torch.randn(param.size()) * noise_factor
            param.add_(noise)
    print(f"Added noise with factor {noise_factor} to model weights.")

## Tune noise hyperparameters

In [15]:
def tune_noise(model, test_loader, noise_factors):
    best_noise_factor = None
    best_accuracy = 0
    for noise_factor in noise_factors:
        original_state_dict = model.state_dict()
        add_noise_to_weights(model, noise_factor)
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels.unsqueeze(1)).sum().item()
        accuracy = 100 * correct / total
        print(f"Noise Factor: {noise_factor}, Accuracy: {accuracy:.2f}%")
        if accuracy > best_accuracy:
            best_noise_factor = noise_factor
            best_accuracy = accuracy
        model.load_state_dict(original_state_dict)
    print(f"Best Noise Factor: {best_noise_factor}")
    print(f"Best Accuracy: {best_accuracy:.2f}%")
    return best_noise_factor

## Train and evaluate the model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
NUM_EPOCHS = 50

# Train the model
train_model(model, criterion, optimizer, train_loader, num_epochs=NUM_EPOCHS)

# Evaluate the model
evaluate_model(model, test_loader)

Epoch [1/50], Loss: 0.3803
Epoch [2/50], Loss: 0.1816
Epoch [3/50], Loss: 0.1658
Epoch [4/50], Loss: 0.1526
Epoch [5/50], Loss: 0.1411
Epoch [6/50], Loss: 0.1331
Epoch [7/50], Loss: 0.1234
Epoch [8/50], Loss: 0.1135
Epoch [9/50], Loss: 0.1067
Epoch [10/50], Loss: 0.1031
Epoch [11/50], Loss: 0.0949
Epoch [12/50], Loss: 0.0923
Epoch [13/50], Loss: 0.0851
Epoch [14/50], Loss: 0.0863
Epoch [15/50], Loss: 0.0801
Epoch [16/50], Loss: 0.0773
Epoch [17/50], Loss: 0.0750
Epoch [18/50], Loss: 0.0765
Epoch [19/50], Loss: 0.0705
Epoch [20/50], Loss: 0.0706
Epoch [21/50], Loss: 0.0707
Epoch [22/50], Loss: 0.0692
Epoch [23/50], Loss: 0.0659
Epoch [24/50], Loss: 0.0620
Epoch [25/50], Loss: 0.0631
Epoch [26/50], Loss: 0.0600
Epoch [27/50], Loss: 0.0645
Epoch [28/50], Loss: 0.0572
Epoch [29/50], Loss: 0.0561
Epoch [30/50], Loss: 0.0568
Epoch [31/50], Loss: 0.0610
Epoch [32/50], Loss: 0.0546
Epoch [33/50], Loss: 0.0564
Epoch [34/50], Loss: 0.0558
Epoch [35/50], Loss: 0.0514
Epoch [36/50], Loss: 0.0510
E

## Find the best noise factor

In [17]:
noise_factors = [0.001, 0.01, 0.1, 0.5, 1.0]
best_noise_factor = tune_noise(model, test_loader, noise_factors)
print('Best Noise Factor: ', best_noise_factor)

Added noise with factor 0.001 to model weights.
Noise Factor: 0.001, Accuracy: 2065.67%
Added noise with factor 0.01 to model weights.
Noise Factor: 0.01, Accuracy: 2076.98%
Added noise with factor 0.1 to model weights.
Noise Factor: 0.1, Accuracy: 1924.11%
Added noise with factor 0.5 to model weights.
Noise Factor: 0.5, Accuracy: 2279.06%
Added noise with factor 1.0 to model weights.
Noise Factor: 1.0, Accuracy: 53.51%
Best Noise Factor: 0.5
Best Accuracy: 2279.06%
Best Noise Factor:  0.5


## Evaluate model with the best noise factor

In [18]:
def test_with_best_noise(model, test_loader, best_noise_factor):
    original_state_dict = model.state_dict()
    add_noise_to_weights(model, best_noise_factor)
    evaluate_model(model, test_loader)
    model.load_state_dict(original_state_dict)
    print("Restored original model weights.")

test_with_best_noise(model, test_loader, best_noise_factor)

Added noise with factor 0.5 to model weights.
Accuracy: 565.17%
Restored original model weights.


## Add different types of noise to model weights

In [19]:
def add_salt_and_pepper_noise(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            mask = torch.rand(param.size()) < noise_factor
            param[mask] = torch.rand(mask.sum().item())
    print(f"Added salt and pepper noise with factor {noise_factor} to model weights.")

def add_gaussian_noise(model, noise_factor):
    with torch.no_grad():
        for param in model.parameters():
            noise = torch.randn(param.size()) * noise_factor
            param.add_(noise)
    print(f"Added Gaussian noise with factor {noise_factor} to model weights.")

## Tune different noise hyperparameters

In [20]:
def tune_different_noises(model, test_loader, noise_factors, noise_type):
    best_noise_factor = None
    best_accuracy = 0
    for noise_factor in noise_factors:
        original_state_dict = model.state_dict()
        if noise_type == 'salt_and_pepper':
            add_salt_and_pepper_noise(model, noise_factor)
        elif noise_type == 'gaussian':
            add_gaussian_noise(model, noise_factor)
        else:
            add_noise_to_weights(model, noise_factor)
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels.unsqueeze(1)).sum().item()
        accuracy = 100 * correct / total
        print(f"Noise Type: {noise_type}, Noise Factor: {noise_factor}, Accuracy: {accuracy:.2f}%")
        if accuracy > best_accuracy:
            best_noise_factor = noise_factor
            best_accuracy = accuracy
        model.load_state_dict(original_state_dict)
    print(f"Best Noise Type: {noise_type}, Best Noise Factor: {best_noise_factor}")
    print(f"Best Accuracy: {best_accuracy:.2f}%")
    return best_noise_factor

## Find the best noise factor for different noise types

In [21]:
noise_factors = [0.001, 0.01, 0.1, 0.5, 1.0]
noise_types = ['salt_and_pepper', 'gaussian', 'default']
best_noise_factors = {}
for noise_type in noise_types:
    best_noise_factor = tune_different_noises(model, test_loader, noise_factors, noise_type)
    best_noise_factors[noise_type] = best_noise_factor
print('Best Noise Factors: ', best_noise_factors)

Added salt and pepper noise with factor 0.001 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.001, Accuracy: 538.53%
Added salt and pepper noise with factor 0.01 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.01, Accuracy: 558.30%
Added salt and pepper noise with factor 0.1 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.1, Accuracy: 766.03%
Added salt and pepper noise with factor 0.5 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 0.5, Accuracy: 3609.14%
Added salt and pepper noise with factor 1.0 to model weights.
Noise Type: salt_and_pepper, Noise Factor: 1.0, Accuracy: 3609.14%
Best Noise Type: salt_and_pepper, Best Noise Factor: 0.5
Best Accuracy: 3609.14%
Added Gaussian noise with factor 0.001 to model weights.
Noise Type: gaussian, Noise Factor: 0.001, Accuracy: 3609.14%
Added Gaussian noise with factor 0.01 to model weights.
Noise Type: gaussian, Noise Factor: 0.01, Accuracy: 3609.14%
Added Gaussian noise with factor 0.

## Evaluate model with the best noise factors

In [22]:
def test_with_best_noises(model, test_loader, best_noise_factors):
    for noise_type, noise_factor in best_noise_factors.items():
        print(f"Testing with best noise type: {noise_type}, factor: {noise_factor}")
        original_state_dict = model.state_dict()
        if noise_type == 'salt_and_pepper':
            add_salt_and_pepper_noise(model, noise_factor)
        elif noise_type == 'gaussian':
            add_gaussian_noise(model, noise_factor)
        else:
            add_noise_to_weights(model, noise_factor)
        evaluate_model(model, test_loader)
        model.load_state_dict(original_state_dict)
        print("Restored original model weights.")

test_with_best_noises(model, test_loader, best_noise_factors)

Testing with best noise type: salt_and_pepper, factor: 0.5
Added salt and pepper noise with factor 0.5 to model weights.
Accuracy: 3609.14%
Restored original model weights.
Testing with best noise type: gaussian, factor: 0.001
Added Gaussian noise with factor 0.001 to model weights.
Accuracy: 3609.14%
Restored original model weights.
Testing with best noise type: default, factor: 0.001
Added noise with factor 0.001 to model weights.
Accuracy: 3609.14%
Restored original model weights.
